In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import gzip

In [2]:
import sys
sys.path.append("/Users/casey/git/mumodo/packages")

In [3]:
from mumodo import mumodoIO as mm

In [4]:
import os

folders = os.listdir('../cleaned')

In [5]:
data = {}

for f in folders:
    pid = f.split("_")[0]
    sf = mm.open_streamframe_from_xiofile(os.path.join('../cleaned/', f, 'log.xio.gz'), '/InproTK/', window_size=-1)
    ph = sf['PHASE']
    sf = sf['TASK']
    ph = ph.dropna()
    sf = sf.dropna()
    data[pid] = sf
    data[pid+'_phase'] = ph
    

opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing


In [6]:
def get_task_frame_pairs2(sf):
    f = sf[sf.str.contains("frame")]
    frames = pd.DataFrame(f.values , columns = ['frames'])
    frames.index = f.index
    t = sf[sf.str.contains("task")]
    tasks = pd.DataFrame(t.values, columns = ['tasks'])
    tasks.index = t.index
    d = sf[sf.str.contains("domain")]
    domains = pd.DataFrame(d.values, columns = ['domains'])
    domains.index = d.index
    merged = pd.merge(frames, domains, how="outer", left_index=True, right_index=True)
    merged = pd.merge(merged, tasks, how="outer", left_index=True, right_index=True)
    merged = merged.ffill()
    print merged[:10]

In [32]:
class Task:
    def __init__(self):
        self.frame = {}
        self.task = {}
        self.start = None
        self.end = None
        self.domain = None
        self.intent = None
        self.phase = None
    def elapsed(self):
        return (self.end - self.start) / 1000

In [57]:
import operator

def get_phase_boundaries(sf):
    phase_ids = {"non":0.0,"incr":0.0,"adapt":0.0}
    for i,row in sf.iteritems():
        if "phase:" in row:
            row = row.split(":")
            phase_ids[row[1]] = i
    return sorted(phase_ids.items(), key=operator.itemgetter(1))
    
def determine_phase(i, phases):
    phase = None
    for p,j in phases:
        if int(i) > int(j):
            phase = p
    return phase

In [55]:
def get_task_frame_pairs(sf, phases):
    current = Task()
    tasks = []
    for i,row in sf.iteritems():
        phase = determine_phase(i,phases)
        row = row.split(":")
        if len(row) < 2: continue
        elif row[1] == 'intent':
            current.intent = row[2]
            current.end = i
        elif row[0] == 'frame':
            current.frame[row[1]] = row[2]
        elif row[0] == 'domain':
            tasks.append(current)
            current = Task()
            current.domain = row[1]
            current.start = i
        elif row[0] == 'task':
            current.task[row[1]] = row[2]
    return tasks

In [56]:
ind = 9
for pid in data:
    if "phase" in pid: continue
    phases = get_phase_boundaries(data[pid+'_phase'])
    tasks = get_task_frame_pairs(data[pid], phases)
    print tasks[ind].domain, tasks[ind].intent
    print tasks[ind].elapsed()
    print tasks[ind].task
    print tasks[ind].frame
    break

41761 non
41761 non
41761 non
41763 non
41763 non
64534 non
64535 non
64535 non
64535 non
64536 non
95256 non
95258 non
95259 non
95263 non
95263 non
95263 non
123246 non
123246 non
123246 non
123249 non
123250 non
123250 non
158518 non
158518 non
158518 non
158520 non
158520 non
178453 non
178453 non
178455 non
178455 non
178455 non
201432 non
201432 non
201433 non
201437 non
201437 non
218863 non
218864 non
218868 non
218868 non
218868 non
289388 incr
289388 incr
289388 incr
289392 incr
289392 incr
289392 incr
377533 incr
377534 incr
377534 incr
377537 incr
377537 incr
377537 incr
397072 incr
397072 incr
397072 incr
397072 incr
397072 incr
397072 incr
409697 incr
409697 incr
409697 incr
409699 incr
409699 incr
409699 incr
423067 incr
423068 incr
423068 incr
423071 incr
423071 incr
430590 incr
430594 incr
430595 incr
430595 incr
430595 incr
441454 incr
441454 incr
441454 incr
441455 incr
441456 incr
441456 incr
452002 incr
452003 incr
452003 incr
452003 incr
452003 incr
452004 incr
48